In [5]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
seed = 0
np.random.seed(seed)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'whitegrid')
from datetime import date, timedelta
#!pip install twint
import twint
import nest_asyncio
nest_asyncio.apply()
import datetime as dt
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from wordcloud import WordCloud

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

In [7]:
c = twint.Config()
c.Username = "detikcom"
c.Search = '"hoax" lang:id'
c.Limit = 10000
c.Store_csv = True
c.Output = 'PALSU.csv'
c.Since = "2021-01-01"
c.Until = "2022-01-01"
twint.run.Search(c)

1476460783945060352 2021-12-30 14:50:56 +0700 <detikcom> Sunny Dahye melapor dugaan pengancaman dan penyebar hoax ke Polres Jakarta Utara. Kini, Sunny Dahye akhirnya menemukan penyebar hoax itu. #SunnyDahye   https://t.co/YQgAvvhREI
1475710693865312256 2021-12-28 13:10:21 +0700 <detikcom> Baru-baru ini geger varian Corona 'Delmicron', gabungan varian Delta dan Omicron. Sudah dipastikan hoax, ternyata begini asal-muasal muncul kabar Delmicron.  https://t.co/HEMmAt6Dj2
1475425582712340483 2021-12-27 18:17:25 +0700 <detikcom> Dunia dihebohkan dengan rumor tentang varian Delmicron yang disebut gabungan antara varian Delta dan Omicron. Namun ternyata, varian ini adalah hoax.  https://t.co/YvYb4UEYTf
1475425168679972867 2021-12-27 18:15:46 +0700 <detikcom> Banyak yang menyangka Delmicron adalah varian Corona baru kombinasi Delta-Omicron. Nyatanya, varian Delmicron hoax belaka.   https://t.co/R1hv5FqVWm
1475350840047243264 2021-12-27 13:20:25 +0700 <detikcom> Varian Delmicron mendadak jadi pe

In [8]:
tweets_data = pd.read_csv('PALSU.csv')
tweets = tweets_data[['id', 'username', 'tweet']]
tweets

,id,username,tweet
0,1476460783945060352,detikcom,Sunny Dahye melapor dugaan pengancaman dan pen...
1,1475710693865312256,detikcom,"Baru-baru ini geger varian Corona 'Delmicron',..."
2,1475425582712340483,detikcom,Dunia dihebohkan dengan rumor tentang varian D...
3,1475425168679972867,detikcom,Banyak yang menyangka Delmicron adalah varian ...
4,1475350840047243264,detikcom,"Varian Delmicron mendadak jadi perbincangan, d..."
5,1475228308308729859,detikcom,"Varian Delmicron mendadak jadi perbincangan, d..."
6,1475107984007208967,detikcom,Epidemiolog Universitas Griffith Dicky Budiman...
7,1471908044216930309,detikcom,Hoax menerpa Menteri Agama Yaqut Cholil Qoumas...
8,1469314300451950594,detikcom,Dipastikan hoax! BNI tidak pernah memungut bia...
9,1468767868452020224,detikcom,Santer beredar informasi 4 kasus varian Omicro...
